## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("world_weather_analysis/weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,39.79,81,2,4.59,clear sky
1,1,Hasanpur,IN,28.7167,78.2833,53.20,68,2,0.40,clear sky
2,2,Vaini,TO,-21.2000,-175.2000,75.36,94,100,12.66,light rain
3,3,Airai,TL,-8.9266,125.4092,61.11,89,91,1.07,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,39.47,99,4,2.01,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.isnull().sum()

104


City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_travel_cities_df = preferred_cities_df.dropna()
print(len(clean_travel_cities_df))
clean_travel_cities_df.head()

103


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Hithadhoo,MV,-0.6000,73.0833,82.69,72,21,12.66,few clouds
8,8,Butaritari,KI,3.0707,172.7902,81.30,77,63,15.21,light rain
16,16,Simbahan,PH,6.3032,120.5836,80.98,81,98,16.91,overcast clouds
21,21,Coahuayana,MX,18.7333,-103.6833,85.48,27,3,6.62,clear sky
22,22,Atar,MR,20.5169,-13.0499,89.65,10,0,6.91,clear sky


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hithadhoo,MV,82.69,few clouds,-0.6000,73.0833,
8,Butaritari,KI,81.30,light rain,3.0707,172.7902,
16,Simbahan,PH,80.98,overcast clouds,6.3032,120.5836,
21,Coahuayana,MX,85.48,clear sky,18.7333,-103.6833,
22,Atar,MR,89.65,clear sky,20.5169,-13.0499,
40,Tautira,PF,84.22,light rain,-17.7333,-149.1500,
46,Touros,BR,88.70,scattered clouds,-5.1989,-35.4608,
55,Presidencia Roque Saenz Pena,AR,85.60,few clouds,-26.7852,-60.4388,
56,Karratha,AU,84.00,few clouds,-20.7377,116.8463,
73,Fayaoue,NC,82.00,overcast clouds,-20.6500,166.5333,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hithadhoo,MV,82.69,few clouds,-0.6000,73.0833,Scoop Guest House
8,Butaritari,KI,81.30,light rain,3.0707,172.7902,Isles Sunset Lodge
21,Coahuayana,MX,85.48,clear sky,18.7333,-103.6833,El rranchito michuacan de ocampos
22,Atar,MR,89.65,clear sky,20.5169,-13.0499,Oumou Elghoura
40,Tautira,PF,84.22,light rain,-17.7333,-149.1500,Pension Oaoa
...,...,...,...,...,...,...,...
672,Ilhabela,BR,82.44,overcast clouds,-23.7781,-45.3581,Ilha Flat Hotel
677,Vila Velha,BR,85.95,few clouds,-20.3297,-40.2925,Quality Suites Vila Velha
679,Loandjili,CG,80.71,few clouds,-4.7561,11.8578,Résidence ARCHANGE
682,Sao Jose Da Coroa Grande,BR,84.22,overcast clouds,-8.8978,-35.1478,Crôa Mares Hotel


In [24]:
print(len(clean_hotel_df))
clean_hotel_df.dtypes
clean_hotel_df.isna().sum()

99


City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [25]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hithadhoo,MV,82.69,few clouds,-0.6000,73.0833,Scoop Guest House
8,Butaritari,KI,81.30,light rain,3.0707,172.7902,Isles Sunset Lodge
21,Coahuayana,MX,85.48,clear sky,18.7333,-103.6833,El rranchito michuacan de ocampos
22,Atar,MR,89.65,clear sky,20.5169,-13.0499,Oumou Elghoura
40,Tautira,PF,84.22,light rain,-17.7333,-149.1500,Pension Oaoa
...,...,...,...,...,...,...,...
672,Ilhabela,BR,82.44,overcast clouds,-23.7781,-45.3581,Ilha Flat Hotel
677,Vila Velha,BR,85.95,few clouds,-20.3297,-40.2925,Quality Suites Vila Velha
679,Loandjili,CG,80.71,few clouds,-4.7561,11.8578,Résidence ARCHANGE
682,Sao Jose Da Coroa Grande,BR,84.22,overcast clouds,-8.8978,-35.1478,Crôa Mares Hotel


In [26]:
# 8a. Create the output File (CSV)
output_data_file = ("World_Weather_Analysis/weather_data/WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))